In [ ]:
import pandas as pd

column_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes",
    "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
    "num_compromised","root_shell","su_attempted","num_root","num_file_creations",
    "num_shells","num_access_files","num_outbound_cmds","is_host_login",
    "is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
    "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
    "srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack","level"
]

df = pd.read_csv("KDDTest-21.txt", names=column_names)
df.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,13,tcp,telnet,SF,118,2425,0,0,0,0,...,0.38,0.12,0.04,0.0,0.00,0.00,0.12,0.3,guess_passwd,2
1,0,udp,private,SF,44,0,0,0,0,0,...,1.00,0.01,0.01,0.0,0.00,0.00,0.00,0.0,snmpguess,12
2,0,tcp,telnet,S3,0,44,0,0,0,0,...,0.31,0.61,0.00,0.0,0.21,0.68,0.60,0.0,processtable,18
3,0,udp,private,SF,53,55,0,0,0,0,...,1.00,0.00,0.87,0.0,0.00,0.00,0.00,0.0,normal,17
4,0,tcp,private,SH,0,0,0,0,0,0,...,0.06,1.00,1.00,0.0,1.00,1.00,0.00,0.0,nmap,17


In [ ]:
!pip install scikit-learn xgboost


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Load the dataset
column_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes",
    "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
    "num_compromised","root_shell","su_attempted","num_root","num_file_creations",
    "num_shells","num_access_files","num_outbound_cmds","is_host_login",
    "is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
    "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
    "srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack","level"
]

df = pd.read_csv("KDDTest-21.txt", names=column_names)

print(df.head())
print(df['attack'].value_counts())

# 2. Convert categorical columns to numeric
categorical_cols = ["protocol_type", "service", "flag"]
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

# 3. Separate features and labels
X = df.drop(columns=["attack", "level"])
y = df["attack"]

# Optionally convert attack types into normal/anomaly (binary classification)
y = y.apply(lambda x: "normal" if x == "normal" else "anomaly")

# 4. Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


   duration protocol_type  service flag  src_bytes  dst_bytes  land  \
0        13           tcp   telnet   SF        118       2425     0   
1         0           udp  private   SF         44          0     0   
2         0           tcp   telnet   S3          0         44     0   
3         0           udp  private   SF         53         55     0   
4         0           tcp  private   SH          0          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.38   
1               0       0    0  ...                    1.00   
2               0       0    0  ...                    0.31   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    0.06   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.12                         0.04   
1                    0.01                         0.01   
2                   

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1907   21]
 [  33  409]]
              precision    recall  f1-score   support

     anomaly       0.98      0.99      0.99      1928
      normal       0.95      0.93      0.94       442

    accuracy                           0.98      2370
   macro avg       0.97      0.96      0.96      2370
weighted avg       0.98      0.98      0.98      2370



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Split dataset into training/testing (use X_scaled and y from preprocessing)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# ✅ Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy: {accuracy*100:.2f}%")

# ✅ Precision, Recall, F1-score
precision = precision_score(y_test, y_pred, pos_label="anomaly")  # use pos_label="anomaly" for binary classification
recall = recall_score(y_test, y_pred, pos_label="anomaly")
f1 = f1_score(y_test, y_pred, pos_label="anomaly")

print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1-score: {f1:.4f}")

# ✅ Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=["normal", "anomaly"])
print("\n✅ Confusion Matrix:")
print(cm)

# (Optional) More detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=["normal", "anomaly"]))


✅ Accuracy: 97.17%
✅ Precision: 0.9805
✅ Recall: 0.9851
✅ F1-score: 0.9828

✅ Confusion Matrix:
[[ 392   38]
 [  29 1911]]

Detailed Classification Report:
              precision    recall  f1-score   support

      normal       0.98      0.99      0.98      1940
     anomaly       0.93      0.91      0.92       430

    accuracy                           0.97      2370
   macro avg       0.96      0.95      0.95      2370
weighted avg       0.97      0.97      0.97      2370



In [ ]:
import joblib

joblib.dump(model, "model.joblib")
joblib.dump(scaler, "preprocess.joblib")


['preprocess.joblib']